In [1]:
import os, sys
import re
import json
import glob

import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from collections import defaultdict
from datetime import datetime
from itertools import cycle
import numpy as np
import matplotlib.pyplot as plt

rpath = os.path.abspath('./..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils
from src.utils import get_messages_from_channel,from_msg_get_replies_text



In [2]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [3]:
data_loader = SlackDataLoader(path='./../dataset/')
channels_data = data_loader.get_channels()

In [8]:
import os
import json
import pandas as pd

def get_total_messages_and_replies(channels_data):
    '''
    Get the total messages and replies for each channel
    '''
    all_messages = []
    all_replies = []

    for channel in channels_data:
        channel_path = f"../dataset/{channel['name']}"
        channel_messages = get_messages_from_channel(channel_path)

        # Extract replies for each message
        channel_messages["replies"] = channel_messages.apply(lambda msg: from_msg_get_replies_text(msg), axis=1)

        # Append messages and replies to respective lists
        all_messages.extend(channel_messages["text"].tolist())
        all_replies.extend(channel_messages["replies"].tolist())

    return all_messages, all_replies



all_messages, all_replies = get_total_messages_and_replies(channels_data)

print("All Messages:")
for message in all_messages:
    print(message)
print("\nAll Replies:")
for reply_list in all_replies:
    for reply in reply_list:
        print(reply, "replies")

All Messages:
<!channel> Where exactly can I find the certificate?
Same here
We will be emailing them out early next week
<@U03T89ACUUW> is off this week
Okay thanks
<@U03U9DB7REG> has joined the channel
<@U03UKL27B0R> has joined the channel
<@U03UL5LSTG9> has joined the channel
<@U03TT5KEYCF> has joined the channel
<@U03UYNR4TS4> has joined the channel
<@U03UP7V9Q57> has joined the channel
Hi all,

We're missing 8 people - <https://forms.gle/tYezqzWzwqTwKfBFA>

Help us get to 100%.
Hello Everyone, <!here> I have responded to the feedback form very late but I think it was a great thing to do since it have the potential of making 10 Academy even more interesting for the next batches. Guys <!here>we aren't trainees anymore we are a community we can make 10 academy even more better because we are in this together. SO <@U03UVHCV6KB> <@U03UJKJGRAQ> <@U03U93GNNVB>  <@U03V1AM5TFA>   <!here>I post this challenge you go fill the form and come here and post "I am one of them" ... we are only 8 p

In [10]:
import os
import json
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string

# def get_total_messages_and_replies(channels_data):
#     '''
#     Get the total messages and replies for each channel
#     '''
#     all_messages = []
#     all_replies = []

#     for channel in channels_data:
#         channel_path = f"../dataset/{channel['name']}"
#         channel_messages = get_messages_from_channel(channel_path)

#         channel_messages["replies"] = channel_messages.apply(lambda msg: from_msg_get_replies_text(msg), axis=1)
#         all_messages.extend(channel_messages["text"].tolist())
#         all_replies.extend(channel_messages["replies"].tolist())

#     return all_messages, all_replies

def preprocess_text(text):
    return simple_preprocess(text, deacc=True, min_len=2)

def preprocess_messages(messages):
    return [preprocess_text(message) for message in messages]

all_messages, all_replies = get_total_messages_and_replies(channels_data)

# Preprocess the messages
processed_messages = preprocess_messages(all_messages)

# Create dictionary and corpus
dictionary = corpora.Dictionary(processed_messages)
corpus = [dictionary.doc2bow(message) for message in processed_messages]

# Train the LDA model
num_topics = 10
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Print the top 10 topics
topics = lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=False)
for topic_id, topic in topics:
    print(f"Topic {topic_id + 1}:")
    for word, prob in topic:
        print(f"- {word} (Prob: {prob:.4f})")
    print()

Topic 1:
- joy (Prob: 0.0503)
- home (Prob: 0.0331)
- site (Prob: 0.0252)
- lib (Prob: 0.0251)
- kb (Prob: 0.0229)
- jds (Prob: 0.0228)
- uvhcv (Prob: 0.0221)
- getting (Prob: 0.0212)
- git (Prob: 0.0211)
- essy (Prob: 0.0180)

Topic 2:
- mzcz (Prob: 0.0743)
- uus (Prob: 0.0743)
- does (Prob: 0.0226)
- text (Prob: 0.0216)
- ok (Prob: 0.0215)
- local (Prob: 0.0174)
- submission (Prob: 0.0161)
- my (Prob: 0.0156)
- running (Prob: 0.0156)
- document (Prob: 0.0153)

Topic 3:
- the (Prob: 0.1915)
- has (Prob: 0.1701)
- channel (Prob: 0.1686)
- joined (Prob: 0.1628)
- uur (Prob: 0.0159)
- gulu (Prob: 0.0141)
- file (Prob: 0.0129)
- session (Prob: 0.0117)
- next (Prob: 0.0087)
- challenge (Prob: 0.0087)

Topic 4:
- to (Prob: 0.0547)
- the (Prob: 0.0526)
- you (Prob: 0.0408)
- and (Prob: 0.0254)
- we (Prob: 0.0225)
- it (Prob: 0.0202)
- can (Prob: 0.0184)
- that (Prob: 0.0155)
- have (Prob: 0.0154)
- is (Prob: 0.0150)

Topic 5:
- rx (Prob: 0.1104)
- uun (Prob: 0.1102)
- hi (Prob: 0.0233)
- be 